In [5]:
from selenium import webdriver # 셀레니움과 웹드라이버 부르는 모듈함수 
import time
import pymysql

   
    
# croll 2번씩 내려감
def scroll_dowmn_page2():
    for i in range(2): # adjust integer value for need
        driver.execute_script("window.scrollBy(0, 200)")
        time.sleep(0.5)
        
        
        
        
# 법률 구조 공단 크롤링  - 질문자 데이터
def klac_data_Writer():
    writer = []
    Org = 3

    # 사건 제목 title
    # 상닫번호 num
    # 신청일 data
    # 진행현황 process_state
    # 작성자가 쓴 내용 content

    title = driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[2]/div[1]/h2").text
    num =  driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[2]/div[2]/dl[1]/dd").text
    data = driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[2]/div[2]/dl[2]/dd").text
    #process_state = driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[2]/div[2]/dl[3]/dd").text
    content = driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[2]/div[3]").text
    content = content.replace("\n","")

    writer.append(Org)
    writer.append(num)
    writer.append(title)
    writer.append(content)
    #writer.append(process_state)
    writer.append(data)

    print(*writer)
    insert_inquiry(writer)
    return writer



# 법률 구조 공단 크롤링  - 답변자 데이터
def klac_data_Answer():
    answer =[]

    # 답변일 data2
    # 답변자(변호사)가 쓴 내용 answer_content

    answer_data = driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[3]/div[1]/dl[1]/dd").text 
    num =  driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[2]/div[2]/dl[1]/dd").text

    answer_content =  driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[3]/div[2]").text
    answer_content = answer_content.replace("\n","")

    answer.append(num)
    answer.append(answer_content)
    answer.append(answer_data) # 사건 번호

    print(*answer)
    insert_answer(answer)
    return answer




# AWS DB 연동
def get_db():
    db = pymysql.connect(
        host='ec2-13-125-144-25.ap-northeast-2.compute.amazonaws.com',
        port=3306,
        user='admin',
        passwd='1/zw;GytAwx*',
        db='KLAC',
        charset='utf8'
    )
    return db







# AWS Inquiry TABLE DB에 데이터 넣기
def insert_inquiry(data):
    db = get_db()
    mycursor = db.cursor()

    try:
        sql = "INSERT INTO Inquiry (OrgIndex,Num,Title,content,regDate,Link) VALUES (%s, %s, %s, %s, %s, %s)"
        val = (data[0],data[1],data[2],data[3],data[4],a)
        mycursor.execute(sql, val)

        db.commit()
        db.close()

    except Exception as e:
        print('db insert inquiry error', e)
        db.close()

        
        
        

# AWS ANSWER TABLE DB에 데이터 넣기
def insert_answer(data):
    db = get_db()
    mycursor = db.cursor()

    try:
        sql = "INSERT INTO Answer (Num,content,regDate,Link) VALUES (%s, %s, %s, %s)"
        val = (data[0],data[1],data[2],a)
        mycursor.execute(sql, val)

        db.commit()
        db.close()

    except Exception as e:
        print('db insert inquiry error', e)
        db.close()




# 게시글 수가 100단위 일 때, next page
# [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000] 
arr = []
for i in range(1,31):
    i = 100 * i
    arr.append(i)

    
    
    
    
# ------------ main --------------
driver = webdriver.Chrome('C:\/chromedriver.exe') # 제어하기 위해 변수 선언
driver.maximize_window() ### 창의 해상도 크기 최대화 (전체화면) ###



print("==================================================================================")
print("--------------------로컬 PC에서 Crawling 작업을 시작하겠습니다----------------------")
print("==================================================================================")
print("--------------------------------법률구조공단(KLAC)--------------------------------")
print("작업 내용: 작성 게시글 1일 이전(00:00~23:59) 모든 데이터 추출 및 저장")
print("\n\n")



try :    
    url = "https://www.klac.or.kr/legalstruct/cyberConsultation/selectOpenArticleList.do?boardCode=3"
    driver.get(url)          
    time.sleep(2)
    scroll_dowmn_page2()
    
    board_num = 1
    
    try :
        
        while True :
                       
            # 페이지네이션 1->2->3->10
            for j in range(1,11):
                case = driver.find_element_by_xpath(f"//*[@id='content']/form[1]/div[2]/div/div[4]/a[{j}]").click()


                #게시글 클릭 1~10 
                for i in range(1,11):

                    driver.implicitly_wait(10)
                    case = driver.find_element_by_xpath(f"//*[@id='content']/form[1]/div[2]/div/div[3]/table/tbody/tr[{i}]").click()
                    
                    # 현재 웹 페이지 URL 추출
                    a = driver.current_url
                    print(a)

                    scroll_dowmn_page2()
                
                
                    try :
                        print("현재 진행중인 게시글 Number :",board_num)
                        board_num += 1

                        driver.implicitly_wait(10)
                        print("===============질문 게시글=================")
                        klac_data_Writer() # 법률 구조 공단 크롤링  - 질문자 데이터
                        
                        print("===============답변 게시글=================")
                        klac_data_Answer() # 법률 구조 공단 크롤링  - 답변자 데이터
                        
                        driver.back() # 뒤로가기

                        print("------------------------------------------------------------------")
                        
                        
                        # 1Next page 버튼 클릭
                        if board_num in arr:
                            driver.find_element_by_xpath("//*[@id='content']/form[1]/div[2]/div/div[4]/button[3]").click()

                    except Exception as e :
                        print("데이터 추출 error",e)

                      
    except Exception as e :
        print("게시글 클릭 error",e)

except Exception as e :
    print("데이터 로딩 error",e)

--------------------로컬 PC에서 Crawling 작업을 시작하겠습니다----------------------
--------------------------------법률구조공단(KLAC)--------------------------------
작업 내용: 작성 게시글 1일 이전(00:00~23:59) 모든 데이터 추출 및 저장



https://www.klac.or.kr/legalstruct/cyberConsultation/selectOpenArticleDetail.do?boardCode=3&contentId=US_0000063255&pageIndex=1&searchCnd=0&searchWrd=
현재 진행중인 게시글 Number : 1
===============질문 게시글=================
3 100007035141 돌아가신 아버지소유의 땅과 건물 돌아가신 아버지의 땅과 땅위에 건물이 있는데 땅은 경매되었고 땅위에 건물은 건축물대장의 면적과 실제면적이 틀려서 경매취하가 되었습니다 건물에는 근저당설정권자 2명이 약 1억에 대해서 근저당 설정을해논상태였습니다 저희 유족입장에서는 해당건물을  철거했으면 하는입장입니다 어떻게하는것이 좋을까요? 2021-07-14
===============답변 게시글=================
100007035141 안녕하십니까. 대한법률구조공단 사이버상담을 이용해 주셔서 감사합니다.법률구조공단 사이버 상담을 이용해주셔서 감사합니다건물의 소유권은 아버님이 돌아가신 후 자녀 등 상속인들이 상속한 것으로 보입니다. 그렇다면 건물의 처분에 대해서는 상속인들에게 권한이 있으므로이를 상속인들의 비용으로 철거하는 것은 가능합니다. 다만, 이 사건의 건물은 근저당이 설정되어 있는 건물이므로 근저당의 피담보채무를 모두 변제하기 전건물철거는 여러 법적문제를 야기할 것으로 보이므로 근저당권을 해결한 후 건물을 철거하여야 할 것입니다.다만, 귀하가 서술한 사실관계와 질문의 요지가 불분명하여 구체적인 사실관계에 따

KeyboardInterrupt: 